In [1]:
from bs4 import BeautifulSoup
import requests

In [5]:
def scrape_entry_data(entry_url):
    # Send an HTTP request to the entry URL
    response = requests.get(entry_url)
    if response.status_code != 200:
        print(f"Failed to fetch data for URL: {entry_url}")
        return
    
    soup = BeautifulSoup(response.text, 'html.parser')
    element = soup.find("div", class_="DeathNoticeBoxItem_box-tile-name__iwcTP")
    # Check if the element was found
    if element:
        # Print the HTML code of the found element
        print(element.prettify())

    print(element)
    #print(name_parts)
    #print("First Name:", first_name)
    #print("Last Name:", last_name)
    print("---")

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find('title').text
    name_parts = title.split()
    first_name = name_parts[0]
    last_name = name_parts[1][:-1]

In [ ]:
element = soup.find("div", class_="DeathNoticeBoxItem_box-tile-name__iwcTP")

    # Check if the element was found
    if element:
        # Print the HTML code of the found element
        print(element.prettify())

In [6]:
entry_urls = [
    "https://rip.ie/death-notice/s/all?page=1&start=2023-09-03+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes",
    "https://rip.ie/death-notice/s/all?page=2&start=2023-09-03+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes",
    # Add more entry URLs here
]

In [7]:
for url in entry_urls:
    scrape_entry_data(url)

None
---
None
---


In [ ]:
"""  
    url_meta = soup.find("meta", property='og:url')
    if url_meta:
        url = url_meta.get('content')
    else:
        url = "URL not found"
"""